### REST API


REST (Representational State Transfer) - это архитектурный стиль, набор принципов проектирования взаимодействия компонентов веб приложения в сети

5 принципов REST:
+ Клиент-серверная архитектура
+ Stateless (сервер не хранит состояние между запросами, каждый запрос должен содержать всю информацию для его обработки)
+ Кэширование (кеширование ресурсов для быстродействия)
+ Единый стиль (соответствие семантики http метода смыслу запроса, единый URL для одинаковых сущностей, формат входных и выходных данных и др.)
+ Многоуровневая система (клиента не должно волновать внутреннее устройство сервера)


### Pydantic

1. Аннотация типов - подсказки от IDE, статический анализ кода и др.
2. Скорость - core библиотеки написан на Rust (это как С/C++, но НАМНОГО сложнее)
3. Автогенерация JSON Schema (для Swagger и тд)
4. Кастомизация и большое кол-во встроенных типов, несколько режимов работы
5. Валидация данных

In [ ]:
class User:
    def __init__(self, username: str, age: int, email: str, password: str):
        if not isinstance(username, str):
            raise ValueError("Username must be a string")
        self.username: str = username
        self.age: int = age
        self.email: str = email
        self.password = password
        if not self.validate_age():
            raise ValueError("User must be adult")
        
    def validate_age(self):
        return self.age >= 14
       
        
user = User(username=123, age=15, email="m1803003@edu.misis.ru", password="Pr1Nciple")
print(user.username)

In [ ]:
from pydantic import BaseModel

# Базовое объявление пайдантик модели
class User(BaseModel):
    username: str
    age: int
    email: str
    password: str

In [ ]:
# Обычная инициализация модели
user = User(username=123, age=21, email="m1803003@edu.misis.ru", password="Pr1Nciple")

In [ ]:
print(user)

In [ ]:
# Десериализация из JSON
user = User.model_validate_json(
    '{"username": "teadove", "age": 21, "email": "m1803003@edu.misis.ru", "password": "Pr1Nciple"}'
)

In [ ]:
user

In [ ]:
# Десериализация из словаря
user = User.model_validate(
    {"username": "teadove", "age": 21, "email": "m1803003@edu.misis.ru", "password": "Pr1Nciple"}
)
user

In [ ]:
# Сериализация в json
user.model_dump_json()

In [ ]:
# Сериализация в словарь
user.model_dump()

In [ ]:
import re
from pydantic import BaseModel, validator


class User(BaseModel):
    username: str
    age: int
    email: str
    password: str

    @validator("age")
    @classmethod
    def validate_age(cls, value):
        if value < 18:
            raise ValueError("User must be adult")
        return value

    @validator("email")
    @classmethod
    def validate_email(cls, value):
        if not bool(re.fullmatch(r"[\w.-]+@[\w-]+\.[\w.]+", value)):
            raise ValueError("Email is invalid")
        return value.lower()

    @validator("password")
    @classmethod
    def validate_password(cls, value):
        password_length = len(value)
        if password_length < 8 or password_length > 16:
            raise ValueError("The password must be between 8 and 16 characters long")
        return value
    

# Valid User
valid_user = {"username": "test_name", "age": 24, "email": "NAME@test.gr", "password": "123456789"}
user = User(**valid_user)
print(user)


### FastAPI

In [ ]:
from fastapi import FastAPI
from fastapi.testclient import TestClient # pip install httpx
from curlify2 import Curlify # pip install curlify2

In [ ]:
!pip install httpx

In [ ]:
!pip install curlify2

In [ ]:
### Тестовый клиент fastapi 
# Чтобы _протестировать_ работоспособность нашего приложения можно применять множество способов, но самый простой - тестовый клиент fastapi
app = FastAPI()
# "https://github.com/hello"
@app.get("/hello")
def hello() -> str:
    return "world"

client = TestClient(app)

response = client.get("/hello")
# Assert - ключевое слово, которое проверяет, что выражение далее истинно. Если оно ложно, то вылетит ошибка AssertionError
assert response.status_code == 200
assert response.text == '"world"'

# curlify2.Curlify позволяет превратить request в curl запрос
Curlify(response.request).to_curl()

In [ ]:
# Хедеры
from fastapi import Header

app = FastAPI()

@app.get("/hello")
def hello(user_name: str = Header(...)) -> str:
    return f"Hello {user_name}"

client = TestClient(app)

# Обратите внимание, что хедеры принято писать в Кебаб-Камел-Кейсе (User-Name), 
#  но в фастапи мы принимаем на вход именно снейк_кейс (user_name)
response = client.get("/hello", headers={"User-Name": "Olya"})

assert response.status_code == 200
assert response.text == '"Hello Olya"'

Curlify(response.request).to_curl()

In [ ]:
# Query
from fastapi import Query

app = FastAPI()

@app.get("/hello")
def hello(user_name: str = Query(...), marks: list[str] = Query(...)) -> str:
    return f"Hello {user_name}, your marks are: {','.join(marks)}"

client = TestClient(app)

# Мы можем принимать на вход даже массивы. 
#  Для этого просто требуется передавать ключ несколько раз
response = client.get("/hello?user_name=stepan&marks=4&marks=5&marks=3")

assert response.status_code == 200
assert response.text == '"Hello stepan, your marks are: 4,5,3"'

Curlify(response.request).to_curl()

In [ ]:
# Path
from fastapi import Path

app = FastAPI()

@app.get("/users/{userId}/hello")
def hello(user_id: str = Path(..., alias="userId")) -> str: # alias - второе название у аргумента, чтобы можно было получить путь userId
    return f"Hello {user_id}"

client = TestClient(app)

response = client.get("/users/123/hello")

assert response.status_code == 200
assert response.text == '"Hello 123"'

Curlify(response.request).to_curl()

### Loguru 
Есть множество разных логеров, которые можно использовать.  
Например, встроенный в питон логгер достаточно хорошо выполняет свой задачи.  
Но для простых и небольших проектов идеально подходит loguru:
`pip install loguru`

In [ ]:
!pip install loguru

In [ ]:
from loguru import logger

logger.debug("когда происходит что-то не очень важное")
logger.info("когда происходит что-то важное")
logger.warning("когда происходит не совсем ошибка, но и не нормальное поведение")
logger.error("когда точно происходит ошибка")

In [ ]:
# А еще можно красиво логировать ошибки
try:
    raise Exception(" ошибка:( ")
except Exception:
    logger.exception("exception.raised")

### Мидлвари

Фастапи поддерживает мидлвари - это такие функции, которые выполняются перед и после запросом, могут отфильтровывать запросы, логировать их, писать время выполнения запроса и тд

In [ ]:
import time
from typing import Callable, Awaitable
from fastapi import Request, Response
from loguru import logger

app = FastAPI()

# Фастапи на данный момент поддерживает только 1 тип мидлварей - на http
@app.middleware("http")
async def add_process_time_header(request: Request, call_next: Callable[[Request], Awaitable[Response]]) -> Response:
    # Мидлварь принимаем на вход request (сам запрос), call_next - функция, что возвращает ответ
    #  с ответом мы можем проводить множество операций, например, добавлять хедеры, логировать запросы и тд

    
    t0 = time.time() # Засекаем время
    
    response = await call_next(request)

    elapsed_ms = round((time.time() - t0) * 1000, 2)
    response.headers["X-Process-Time"] = str(elapsed_ms)
    logger.debug("{} {} done in {}ms", request.method, request.scope["route"].path, elapsed_ms)
    
    return response


@app.get("/hello")
def hello() -> str:
    return "world"

client = TestClient(app)

response = client.get("/hello")

assert response.status_code == 200
assert response.text == '"world"'

print(response.headers['X-Process-Time'])
Curlify(response.request).to_curl()

### Depends 

Depends - функция fastapi, которая позволяет вынести логику валидации данных аспектным подходом, давайте сразу разберем на примере

In [ ]:
from fastapi import Depends, Header, HTTPException, status

app = FastAPI()

# Например, мы хотим проверять, что пользователь передал хедер с ключом доступа 
def has_api_key(x_api_key: str = Header(...)) -> None:
    if x_api_key != "42":
        raise HTTPException(status_code=status.HTTP_403_FORBIDDEN, detail="wrong api key")
    
    return x_api_key

    
# Чтобы подключить нашу зависимость, достаточно указать ее в Depends (вызывать НЕ надо)
@app.get("/hello")
def hello(_ = Depends(has_api_key)) -> str:
    return "world"

client = TestClient(app)

response = client.get("/hello", headers={"X-Api-Key": "10"})

assert response.status_code == 403
assert response.json() == {"detail":"wrong api key"}

response = client.get("/hello", headers={"X-Api-Key": "42"})
print(response.json())

assert response.status_code == 200
assert response.text == '"world"'

Curlify(response.request).to_curl()

Также, Depends можно использовать и для получения аргументов

В данном примере парсится JWT токен, который представляет собой _подписанный_ JSON словарик. Данный подход часто используется в бекенд разработке, поэтому настоятельно советуем его подробно изучить: https://jwt.io/

In [ ]:
!pip uninstall pyjwt

In [ ]:
from fastapi import Depends, Header, HTTPException, status
from typing import Any
import jwt

app = FastAPI()


def x_id_token(x_id_token: str = Header(...)) -> dict[str, Any]:
    try:
        # "secret" - это строчка, с которой мы _подписали_ токен. В идеале она должна лежать где-то в безопасном месте
        # decode - функция расшифровки токена в словарь
        decoded = jwt.decode(x_id_token, "secret", algorithms=["HS256"])
    except Exception as exc:
        raise HTTPException(status_code=status.HTTP_403_FORBIDDEN, detail="bad token") from exc

    return decoded


# Тут мы просто получаем готовытокен, либо кидаем ошибку 
@app.get("/hello")
def hello(id_token: dict[str, Any] = Depends(x_id_token)) -> str:
    return f"Hello {id_token['username']}"

client = TestClient(app)

response = client.get("/hello")

assert response.status_code == 403

# Первый аргумент - словарь, полезная нагрузка, которая используется в токене
token = jwt.encode({"username": "tainella"}, "secret", algorithm="HS256")
response = client.get("/hello", headers={"X-Id-Token": token})

assert response.status_code == 200
assert response.text == '"Hello tainella"'

Curlify(response.request).to_curl()

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia
from loguru import logger

def get_page_from_wikipedia(title: str) -> str:
    try:
        res = wikipedia.page(title)
        return res
    except wikipedia.exceptions.PageError as e:
        logger.error(e)
        return None
    
logger.add("file_{time}.log")
res = get_page_from_wikipedia("qwertyjunhbgvfdef")

# @app.get("/api/search/{title}")
# def search(title: str) -> str:
if res is not None:
    print(res.title)
    print(res.url)
else:
    print("not found")

## ДЗ.
Написать API для получения страниц в википедии.

+ Добавить логирование

+ \*Запустить с uvicorn и протестировать с фронтендом

+ \*Почитать про CORS :)

"/api/search/{title}" -> {"title": "title", "url": "http://wikipedia.org/...."}